In [1]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization

In [ ]:
import os
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

KEYS_DIR = "keys"
PRIVATE_KEY_PATH = os.path.join(KEYS_DIR, "private_key.pem")
PUBLIC_KEY_PATH = os.path.join(KEYS_DIR, "public_key.pem")

# Ensure the keys directory exists
os.makedirs(KEYS_DIR, exist_ok=True)

def generate_keys():
    """Generate RSA key pair and store them in files if not already stored."""
    if not os.path.exists(PRIVATE_KEY_PATH) or not os.path.exists(PUBLIC_KEY_PATH):
        private_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size=2048
        )
        public_key = private_key.public_key()

        with open(PRIVATE_KEY_PATH, "wb") as private_file:
            private_file.write(private_key.private_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PrivateFormat.PKCS8,
                encryption_algorithm=serialization.NoEncryption()
            ))

        with open(PUBLIC_KEY_PATH, "wb") as public_file:
            public_file.write(public_key.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            ))


def get_private_key():
    """Load the private key from the file."""
    with open(PRIVATE_KEY_PATH, "rb") as key_file:
        return serialization.load_pem_private_key(
            key_file.read(),
            password=None
        )

def get_public_key():
    """Load the public key from the file."""
    with open(PUBLIC_KEY_PATH, "rb") as key_file:
        return serialization.load_pem_public_key(key_file.read())

# Generate keys if not present
generate_keys()


In [2]:
import os
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import base64

# Retrieve AES key from environment or generate a new one
key_env = os.getenv("AES_KEY")  
KEY = base64.b64decode(key_env)  # Decode Base64 key from env variable

def encrypt_AES_CBC(plain_text):
    iv = get_random_bytes(16)  # Generate random IV
    cipher = AES.new(KEY, AES.MODE_CBC, iv)
    encrypted_bytes = cipher.encrypt(pad(plain_text.encode(), AES.block_size))
    return base64.b64encode(iv + encrypted_bytes).decode()  # Encode IV + ciphertext

def decrypt_AES_CBC(encrypted_text):
    encrypted_data = base64.b64decode(encrypted_text)
    iv = encrypted_data[:16]  # Extract IV
    cipher = AES.new(KEY, AES.MODE_CBC, iv)
    decrypted_bytes = unpad(cipher.decrypt(encrypted_data[16:]), AES.block_size)
    return decrypted_bytes.decode()

# Example usage
plain_text = "Hello, AES CBC!"
encrypted_text = encrypt_AES_CBC(plain_text)
decrypted_text = decrypt_AES_CBC(encrypted_text)

print("Plain Text:", plain_text)
print("Encrypted Text:", encrypted_text)
print("Decrypted Text:", decrypted_text)


Plain Text: Hello, AES CBC!
Encrypted Text: 3J3lXsK9Zv5V1yhc3cHweXxLxGVOQjfKBsy9Yvev2RE=
Decrypted Text: Hello, AES CBC!
